In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
import random
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [2]:
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data", header=None, names=['sepal length','sepal width','petal length','petal width','class']) 
df

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [3]:
filtro = df['class'] != 'Iris-versicolor'
df_new = df[filtro]
df_new

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [4]:
le = preprocessing.LabelEncoder()
le.fit(["Iris-setosa", "Iris-virginica"])

X = df_new.iloc[:,[0,2,3]]
Y = le.transform(df_new['class'].values)
Y = Y.reshape(-1,1)


scaler1 = MinMaxScaler(feature_range=(0, 1))
scaler1.fit(X)
X_n = scaler1.transform(X)
X_n = np.array(X_n)

print(X_n.shape,Y.shape)

(100, 3) (100, 1)


In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X_n, Y, test_size = 0.3, random_state = 0)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(70, 3)
(30, 3)
(70, 1)
(30, 1)


In [6]:
def fit(x, y, epochs):
    x = x
    y = y

    weights = np.zeros(x.shape[1])
    train_accuracies=[]
    losses=[]
    bias = 0

    for i in range(epochs):
        x_dot_weights = np.matmul(weights, x.transpose()) + bias
        pred = _sigmoid(x_dot_weights)
        loss = compute_loss(y, pred)
        error_w, error_b = compute_gradients(x, y, pred)
        weights,bias = update_model_parameters(weights,bias,error_w, error_b)

        pred_to_class = [1 if p > 0.5 else 0 for p in pred]
        train_accuracies.append(accuracy_score(y, pred_to_class))
        losses.append(loss)

    return weights,bias

def _sigmoid(x):
    return np.array([_sigmoid_function(value) for value in x])

def _sigmoid_function(x):
    if x >= 0:
        z = np.exp(-x)
        return 1 / (1 + z)
    else:
        z = np.exp(x)
        return z / (1 + z)

def compute_loss(y_true, y_pred):
    # binary cross entropy
    y_zero_loss = y_true * np.log(y_pred + 1e-9)
    y_one_loss = (1-y_true) * np.log(1 - y_pred + 1e-9)
    return -np.mean(y_zero_loss + y_one_loss)

def compute_gradients(x, y_true, y_pred):
    # derivative of binary cross entropy
    difference =  y_pred - y_true
    gradient_b = np.mean(difference)
    gradients_w = np.matmul(x.transpose(), difference)
    gradients_w = np.array([np.mean(grad) for grad in gradients_w])

    return gradients_w, gradient_b

def update_model_parameters(weights,bias,error_w, error_b):
    weights = weights - 0.1 * error_w
    bias = bias - 0.1 * error_b
    return weights,bias

def predict(weights,bias, x):
    x_dot_weights = np.matmul(x, weights.transpose()) + bias
    probabilities = _sigmoid(x_dot_weights)
    return [1 if p > 0.5 else 0 for p in probabilities]

In [33]:
weights,bias = fit(x=X_train,y=Y_train,epochs=50)
theta = np.hstack((bias,weights))
print(theta)

[-1.411454   -4.51100862 18.52228766 22.6563801 ]


In [34]:
X_test_stack = np.hstack((np.ones((X_test.shape[0],1)), X_test))
Y_pred = np.dot(X_test_stack,theta)
print(Y_pred)

[ 2.42707542e+00  3.22359000e+01 -2.68501029e-02  3.09142479e+01
  2.66998011e+01  3.16071664e+01  9.84041006e-01  2.43742454e+01
  3.07251309e+01  2.95349127e+01  2.65718050e+01  2.65726644e+01
  2.92785467e+01 -1.09751692e+00  2.25106670e-01  7.89655339e-01
 -8.43355545e-01  1.73146658e+00 -7.15359385e-01  6.63004361e-01
  4.31510710e+00  2.84633864e+01  7.26329850e-01  2.71394177e+01
  1.10579702e+00 -1.50810716e-01  1.35640861e+00  2.55033428e+01
  3.02225625e+01  2.41856141e+01]


In [35]:
y_PRED2 = predict(weights,bias,X_test)
print(np.asarray(y_PRED2))
print(Y_test.T)

[1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1]
[[0 1 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 1]]


In [36]:
mean_squared_error(Y_test,y_PRED2)

0.3333333333333333